In [1]:
!pip install keras

In [2]:
!pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! mkdir ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/Colab_Notebooks/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d kutaykutlu/forest-fire

100% 2.46G/2.46G [02:06<00:00, 21.3MB/s]
100% 2.46G/2.46G [02:06<00:00, 20.9MB/s]


In [ ]:
!unzip forest-fire.zip

Streaming output truncated to the last 5000 lines.
  inflating: train-smoke/008734.jpg  
  inflating: train-smoke/008735.jpg  
  inflating: train-smoke/008736.jpg  
  inflating: train-smoke/008737.jpg  
  inflating: train-smoke/008738.jpg  
  inflating: train-smoke/008739.jpg  
  inflating: train-smoke/008740.jpg  
  inflating: train-smoke/008741.jpg  
  inflating: train-smoke/008742.jpg  
  inflating: train-smoke/008743.jpg  
  inflating: train-smoke/008744.jpg  
  inflating: train-smoke/008745.jpg  
  inflating: train-smoke/008746.jpg  
  inflating: train-smoke/008747.jpg  
  inflating: train-smoke/008748.jpg  
  inflating: train-smoke/008749.jpg  
  inflating: train-smoke/008750.jpg  
  inflating: train-smoke/008751.jpg  
  inflating: train-smoke/008752.jpg  
  inflating: train-smoke/008753.jpg  
  inflating: train-smoke/008754.jpg  
  inflating: train-smoke/008755.jpg  
  inflating: train-smoke/008756.jpg  
  inflating: train-smoke/008757.jpg  
  inflating: train-smoke/008758.jpg  

In [ ]:
#making a folder
!mkdir training_data
import shutil
original = r'/content/train-smoke'
target = r'/content/training_data'
shutil.move(original, target)
original = r'/content/train_fire'
shutil.move(original, target)

'/content/training_data/train_fire'

In [ ]:

#image augmentation
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size=32

training_datagenarator= ImageDataGenerator(rescale=1./255,horizontal_flip=True,
    vertical_flip=True,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2,validation_split=0.1)

In [ ]:
# import os
# dir = os.listdir('/content/forest_dataset/training_data')
# print(dir)
train=training_datagenarator.flow_from_directory('/content/training_data',
                                                target_size=(224, 224),color_mode='rgb',
                                       class_mode='binary', batch_size=batch_size,subset='training')

validation=training_datagenarator.flow_from_directory('/content/training_data',
                                                target_size=(224, 224),color_mode='rgb',
                                       class_mode='binary', batch_size=batch_size,subset='validation')

Found 12360 images belonging to 2 classes.
Found 1373 images belonging to 2 classes.


In [ ]:
train.class_indices

{'train-smoke': 0, 'train_fire': 1}

In [ ]:
from keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LSTM
from tensorflow.keras.applications import VGG16

# Define a CNN model
cnn=tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',activation='relu',input_shape=[224,224,3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))
cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3,padding='same',activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))
cnn.add(tf.keras.layers.Conv2D(filters=256,kernel_size=3,padding='same',activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=128,activation='relu'))
cnn.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

# # Define a VGG16 model
def create_vgg16_model():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers:
        layer.trainable = False
    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='softmax'))
    return model
# Creating FFN model
def create_ffn_model():
    model = Sequential()
    model.add(Flatten(input_shape=(224, 224, 3)))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model


In [ ]:
#train the ffn model 94.46
ffn_model = create_ffn_model()
ffn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
ffn_model.fit(train, epochs=1, validation_data=validation)

387/387 [==============================] - 537s 1s/step - loss: 1.8544 - accuracy: 0.9430 - val_loss: 0.2784 - val_accuracy: 0.9774


In [ ]:
# Train the CNN model
#98.20 accuracy
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn.fit(train,validation_data=validation,epochs=1,steps_per_epoch=train.samples//batch_size,validation_steps=validation.samples//batch_size
                 )

386/386 [==============================] - 549s 1s/step - loss: 0.0752 - accuracy: 0.9821 - val_loss: 0.0113 - val_accuracy: 0.9963


In [ ]:
# Train the VGG16 model
# accuracy 91.75
vgg16_model = create_vgg16_model()
vgg16_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
vgg16_model.fit(train, epochs=1,validation_data=validation)


387/387 [==============================] - 528s 1s/step - loss: nan - accuracy: 0.9178 - val_loss: nan - val_accuracy: 0.9199


In [ ]:
#accuracy 98.34
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
# Train the model
model.fit(train, epochs=1, validation_data=validation)

387/387 [==============================] - 516s 1s/step - loss: 0.0512 - accuracy: 0.9834 - val_loss: 0.0151 - val_accuracy: 0.9949


In [ ]:
############################ Creating a Ensemble Model ##############################

from tensorflow.keras.layers import Input, concatenate
# from tensorflow.keras.models import Model

# Define the input layer
input_layer = Input(shape=(224, 224, 3))

# Get the output from each model
cnn_output = cnn(input_layer)
ffn = ffn_model(input_layer)
#v3_output = model(input_layer)
vgg16_output = vgg16_model(input_layer)

# Concatenate the outputs
merged = concatenate([ffn,cnn_output, vgg16_output])

# Add a fully connected layer
ensemble_output = Dense(10, activation='sigmoid')(merged)

# Define the ensemble model
ensemble_model = Model(inputs=input_layer, outputs=ensemble_output)
ensemble_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the ensemble model
ensemble_model.fit(train, epochs=1,
                   validation_data=validation
                   )

# Evaluate the ensemble model on the test set
# ensemble_model.evaluate(test_)

387/387 [==============================] - 533s 1s/step - loss: nan - accuracy: 0.9197 - val_loss: nan - val_accuracy: 0.9199


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.utils import load_img,img_to_array
path = '/content/training_data/train-smoke/000458.jpg'
img= load_img(path,target_size=(224,224))
i= img_to_array(img)
# i= preprocess_input(i)
input_arr=np.array([i])
input_arr.shape
pred=np.argmax(ensemble_model.predict(input_arr))
print(pred)

#to display image
# plt.show(input_arr[0])
# plt.title("input image")
# plt.show()

1/1 [==============================] - 0s 33ms/step
0
